<a href="https://colab.research.google.com/github/Prasun-Shiwakoti/Anti-CyberBullying/blob/master/model/AI_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
df_new = pd.read_csv('new')

In [ ]:
df_new['length'] = df_new['message'].apply(len)

In [ ]:
messages=df_new['message'].tolist()
label = df_new['label'].tolist()

In [ ]:
vocab_size=16049
embedding_dim=16
max_length=150
truc_type='post'
padding_type='post'
oov_tok="<OOV>"
training_size=10000

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

In [ ]:
training_sentences,testing_sentences,training_labels,testing_labels = train_test_split(messages, label, test_size = 0.3)

In [ ]:
tokenizer = Tokenizer(num_words = vocab_size,oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index= tokenizer.word_index

In [ ]:
training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences,maxlen=max_length,padding=padding_type,truncating=truc_type)

In [ ]:
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length,padding=padding_type,truncating=truc_type)

In [ ]:
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=25)

In [ ]:
from tensorflow.keras.layers import Dropout

In [ ]:
model= tf.keras.Sequential([
tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length=max_length),
tf.keras.layers.GlobalAveragePooling1D(),
tf.keras.layers.Dense(24,activation='relu'),
tf.keras.layers.Dropout(0.5),
tf.keras.layers.Dense(12,activation='relu'),
tf.keras.layers.Dropout(0.5),
tf.keras.layers.Dense(1,activation='sigmoid')
 
])

In [ ]:
 model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
num_epochs=13
history= model.fit(training_padded,training_labels,epochs=num_epochs,validation_data=(testing_padded,testing_labels),callbacks=[early_stop])

Epoch 1/13
361/361 [==============================] - 2s 6ms/step - loss: 0.6494 - accuracy: 0.6675 - val_loss: 0.6345 - val_accuracy: 0.6685
Epoch 2/13
361/361 [==============================] - 2s 6ms/step - loss: 0.6341 - accuracy: 0.6679 - val_loss: 0.6148 - val_accuracy: 0.6685
Epoch 3/13
361/361 [==============================] - 2s 6ms/step - loss: 0.5613 - accuracy: 0.7187 - val_loss: 0.4773 - val_accuracy: 0.8044
Epoch 4/13
361/361 [==============================] - 2s 6ms/step - loss: 0.4440 - accuracy: 0.8064 - val_loss: 0.4114 - val_accuracy: 0.8151
Epoch 5/13
361/361 [==============================] - 2s 6ms/step - loss: 0.3698 - accuracy: 0.8437 - val_loss: 0.4076 - val_accuracy: 0.8244
Epoch 6/13
361/361 [==============================] - 2s 6ms/step - loss: 0.3190 - accuracy: 0.8766 - val_loss: 0.3888 - val_accuracy: 0.8222
Epoch 7/13
361/361 [==============================] - 2s 6ms/step - loss: 0.2715 - accuracy: 0.8992 - val_loss: 0.3999 - val_accuracy: 0.8402
Epoch 

In [ ]:
preds = model.predict_classes(testing_padded)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
0.7548880702748654


In [ ]:
model_loss=pd.DataFrame(model.history.history)
model_loss.plot()

In [ ]:
sentence = ["Muslims are reponsible for all the deaths",]
sequences = tokenizer.texts_to_sequences(sentence)
print(sequences)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=truc_type)
print(model.predict(padded))

[[79, 17, 1, 21, 36, 2, 3481]]
[[0.9357805]]


In [ ]:
import pickle
 
model.save("model.h5")
 
with open("tokenizer.pickle", "wb") as f:
  pickle.dump(tokenizer, f)